In [1]:
%matplotlib tk
#This won't work with cluster
import hyperspy.api as hs
#import matplotlib #Since this is not iPython we cannot use the magic commands
#%matplotlib qt5
#change to qt5 for the installed version on DLS Linuxi
import numpy as np
from math import floor
import pylab as plt
import matplotlib.pyplot as plt
import os
import time
import sys
from pathlib import Path

In [2]:
dir_ = r'C:\Users\gys37319\OneDrive - Nexus365\Data\cm19689-6_Merlin_30keV_ptychography\20180814_MoS2_30kV_pty_tests\20180814 154448'
fn = r'\MOS2_30kV_Pty1_40Mx_15cm_8C'
hdr_fn = dir_ + fn + r'.hdr'
mib_fn = dir_ + fn + r'.mib'


In [3]:
# dictionaries for headers - copied from mib file
format_name = 'hdr'
description = 'hdr file contains the information on how to read\n'
description += 'the mib file with the same name.'
description += '\nThis format does not provide information on the calibration.'
description += '\n You should add this after loading the file.'
full_support = False  # but maybe True
# Recognised file extension
file_extensions = ['hdr']
default_extension = 0
# Writing capabilities
writes = [(1, 0), (1, 1), (1, 2), (2, 0), (2, 1), ]
# ----------------------

# The format only support the followng data types
newline = ('\n', '\r\n')
comment = ';'
sep = '\t'

dtype2keys = {
    'float64': ('float', 8),
    'float32': ('float', 4),
    'uint8': ('unsigned', 1),
    'uint16': ('unsigned', 2),
    'int32': ('signed', 4),
    'int64': ('signed', 8), }

endianess2hdr = {
    '=': 'dont-care',
    '<': 'little-endian',
    '>': 'big-endian'}

# Warning: for selection lists use tuples not lists.
# keys extracted from the hdr file
hdr_keys = {
    # spectrum/image keys
    'width': int,
    'height': int,
    'depth': int,
    'offset': int,
    'data-length': ('1', '2', '4', '8'),
    'data-type': ('signed', 'unsigned', 'float'),
    'byte-order': ('little-endian', 'big-endian', 'dont-care'),
    'record-by': ('image', 'vector', 'dont-care'),
    # X-ray keys
    'ev-per-chan': float,    # usually 5 or 10 eV
    'detector-peak-width-ev': float,  # usually 150 eV
    # HyperSpy-specific keys
    'depth-origin': float,
    'depth-scale': float,
    'depth-units': str,
    'width-origin': float,
    'width-scale': float,
    'width-units': str,
    'height-origin': float,
    'height-scale': float,
    'height-units': str,
    'signal': str,
    # EELS HyperSpy keys
    'collection-angle': float,
    # TEM HyperSpy keys
    'convergence-angle': float,
    'beam-energy': float,
    # EDS HyperSpy keys
    'elevation-angle': float,
    'azimuth-angle': float,
    'live-time': float,
    # From 0.8.5 energy-resolution is deprecated as it is a duplicate of
    # detector-peak-width-ev of the hdr standard format. We keep it here
    # to keep compatibility with hdr file written by HyperSpy < 0.8.4
    'energy-resolution': float,
    'tilt-stage': float,
    'date': str,
    'time': str,
    'title': str,
}

In [4]:
#copied from mib file
def parse_hdr(fp):
    """Parse information from hdr (.hdr) file.
    Accepts file object 'fp. Returns dictionary hdr_info.
    """
    hdr_info = {}
    for line in fp.readlines():
        # skip blank entries
        if any(skip_line in line for skip_line in ('HDR', 'End')):
            continue
        if line[:2] not in newline and line[0] != comment:
            line = line.strip('\r\n')
            if comment in line:
                line = line[:line.find(comment)]
            if sep not in line:
                err = 'Separator in line "%s" is wrong, ' % line
                err += 'it should be a <TAB> ("\\t")'
                raise IOError(err)
            line = [seg.strip() for seg in line.split(sep)]  # now it's a list
            line[0] = line[0].strip(':')  # remove ':' from keys
        hdr_info[line[0]] = line[1]
    # assign values to mandatory keys
    # set the array size of the chip
    # Adding the try argument to accommodate the new hdr formatting
    # as of April 2018
    try:
        if hdr_info['Assembly Size (1X1, 2X2)'] == '1x1':
            hdr_info['width'] = 256
            hdr_info['height'] = 256
        elif hdr_info['Assembly Size (1X1, 2X2)'] == '2x2':
            hdr_info['width'] = 512
            hdr_info['height'] = 512
    except KeyError:
        if hdr_info['Assembly Size (NX1, 2X2)'] == '1x1':
            hdr_info['width'] = 256
            hdr_info['height'] = 256
        elif hdr_info['Assembly Size (NX1, 2X2)'] == '2x2':
            hdr_info['width'] = 512
            hdr_info['height'] = 512

    # convert frames to depth
    hdr_info['depth'] = int(hdr_info['Frames in Acquisition (Number)'])
    # set mib offset
    hdr_info['offset'] = 0
    # set data-type
    hdr_info['data-type'] = 'unsigned'
    # set data-length
    if hdr_info['Counter Depth (number)'] == '1':
        # binary data recorded as 8 bit numbers
        hdr_info['data-length'] = '8'
    else:
        # changes 6 to 8 , 12 to 16 and 24 to 32 bit
        cd_int = int(hdr_info['Counter Depth (number)'])
        hdr_info['data-length'] = str(int((cd_int + cd_int/3)))
    # set byte order
    hdr_info['byte-order'] = 'dont-care'
    # set record by to stack of images
    hdr_info['record-by'] = 'image'

    # set title to file name
    hdr_info['title'] = fp.name.split('\\')[-1]
    # set time and date
    # Adding the try argument to accommodate the new hdr formatting
    # as of April 2018
    try:
        day, month, year_time = hdr_info['Time and Date Stamp (day, mnth, yr, hr, min, s)'].split('/')
        year, time = year_time.split(' ')
        hdr_info['date'] = year + month + day
        hdr_info['time'] = time
    except KeyError:
        day, month, year_time = hdr_info['Time and Date Stamp (yr, mnth, day, hr, min, s)'].split('/')
        year, time = year_time.split(' ')
        hdr_info['date'] = year + month + day
        hdr_info['time'] = time
    return hdr_info

In [5]:
def read_exposure(hdr_info, fp, mmap_mode='c'):
    width = hdr_info['width']
    height = hdr_info['height']
    depth = hdr_info['depth']
    offset = hdr_info['offset']
    data_length = hdr_info['data-length']
    data_type = hdr_info['data-type']
    endian = hdr_info['byte-order']
    record_by = hdr_info['record-by']

    if data_type == 'signed':
        data_type = 'int'
    elif data_type == 'unsigned':
        data_type = 'uint'
    elif data_type == 'float':
        pass
    else:
        raise TypeError('Unknown "data-type" string.')

    # mib data always big-endian
    endian = '>'
    data_type += str(int(data_length))
    data_type = np.dtype(data_type)
    data_type = data_type.newbyteorder(endian)
    #data_type = np.dtype(str)
    # set header number of bits
    hdr_multiplier = (int(data_length)/8)**-1
    hdr_bits = int(384 * hdr_multiplier)

    data = np.memmap(fp,
                     offset=offset,
                     dtype=data_type,
                     mode=mmap_mode)

    if record_by == 'vector':   # spectral image
        size = (height, width, depth)
        data = data.reshape(size)
    elif record_by == 'image':  # stack of images
        width_height = width * height
        size = (depth, height, width)
        # remove headers at the beginning of each frame and reshape
        #data = data.reshape(-1, width_height + hdr_bits)[:, -width_height:].reshape(size)
        # return only headers
        data = data.reshape(-1,  width_height + hdr_bits)[:,71:79]
        data = data [:, ]
    elif record_by == 'dont-care':  # stack of images
        size = (height, width)
        data = data.reshape(size)
    return data

In [6]:
with open(hdr_fn) as f:
    hdr_info = parse_hdr(f)

In [17]:
d = read_exposure(hdr_info,mib_fn)
d.shape

(65536, 8)

In [18]:
d = read_exposure(hdr_info,mib_fn)
exp_time = []
for line in range(d.shape[0]):
    str_list = [chr(d[line][n]) for n in range(d.shape[1])]
    exp_time.append(float(''.join(str_list)))


In [ ]:
plt.plot(exp_time)